In [1]:
# 1. Импорт библиотек
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import re
from collections import defaultdict
import pickle
import os


In [2]:
# 2. Загрузка
df = pd.read_csv('../shared_data/news.csv')

In [3]:
# 3. Загрузка стоп-слов и инициализация стеммера
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')  # именно этот пакет нужен для русского языка


russian_stopwords = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
# 4. Функция предобработки текста
def preprocess_text(text):
    """
    Предобработка текста: очистка, токенизация, удаление стоп-слов и стемминг
    """
    if pd.isna(text):
        return []
    
    # Приведение к нижнему регистру и удаление знаков препинания/цифр
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s]', ' ', str(text))
    text = text.lower()
    
    # Токенизация
    tokens = text.split()
    
    # Удаление стоп-слов и стемминг
    processed_tokens = []
    for token in tokens:
        if token not in russian_stopwords and len(token) > 2:
            stemmed_token = stemmer.stem(token)
            processed_tokens.append(stemmed_token)
    
    return processed_tokens

In [4]:
#4. Усовершенствованная ф-ия предобработки текста (учитывает числовые данные, слова с дефисом)
def improved_preprocess_text(text):
    """
    Улучшенная предобработка с более интеллектуальной токенизацией
    """
    if pd.isna(text):
        return []
    
    # Шаг 1: Более аккуратная очистка
    # Сохраняем числа, дефисы в словах, email-адреса
    text = str(text)
    
    # Шаг 2: Используем интеллектуальную токенизацию nltk
    tokens = word_tokenize(text, language='russian')
    
    # Шаг 3: Фильтрация и нормализация
    processed_tokens = []
    for token in tokens:
        # Приводим к нижнему регистру
        token_lower = token.lower()
        
        # Пропускаем стоп-слова
        if token_lower in russian_stopwords:
            continue
            
        # Пропускаем короткие токены (кроме чисел и специальных случаев)
        if len(token_lower) <= 2 and not token_lower.isdigit():
            continue
            
        # Пропускаем чисто знаки препинания
        if re.match(r'^[^\w\s]+$', token_lower):
            continue
            
        # Стемминг для русских слов
        if re.match(r'[а-яё]+', token_lower):
            stemmed_token = stemmer.stem(token_lower)
            processed_tokens.append(stemmed_token)
        else:
            # Для английских слов, чисел, специальных терминов оставляем как есть
            processed_tokens.append(token_lower)
    
    return processed_tokens

In [5]:
# 5. Построение инвертированного индекса
def build_inverted_index(df, text_columns=['title', 'text']):
    """
    Построение инвертированного индекса из DataFrame
    """
    inverted_index = defaultdict(dict)
    doc_lengths = {}
    
    for idx, row in df.iterrows():
        # Объединяем текст из указанных колонок
        combined_text = ' '.join(str(row[col]) for col in text_columns if col in row and pd.notna(row[col]))
        
        # Предобработка текста
        tokens = improved_preprocess_text(combined_text) #Функция заменена на improved
        doc_lengths[idx] = len(tokens)
        
        # Подсчет TF (Term Frequency) для документа
        term_freq = defaultdict(int)
        for token in tokens:
            term_freq[token] += 1
        
        # Добавление в инвертированный индекс
        for token, freq in term_freq.items():
            inverted_index[token][idx] = freq
    
    return inverted_index, doc_lengths

In [6]:
# 6. Функции для сохранения и загрузки индекса
def save_index(inverted_index, doc_lengths, filename='../shared_data/search_index.pkl'):
    """
    Сохранение индекса в файл
    """
    index_data = {
        'inverted_index': dict(inverted_index),
        'doc_lengths': doc_lengths
    }
    with open(filename, 'wb') as f:
        pickle.dump(index_data, f)
    print(f"Индекс сохранен в {filename}")

def load_index(filename='../shared_data/search_index.pkl'):
    """
    Загрузка индекса из файла
    """
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            index_data = pickle.load(f)
        print(f"Индекс загружен из {filename}")
        return defaultdict(dict, index_data['inverted_index']), index_data['doc_lengths']
    else:
        print(f"Файл {filename} не найден")
        return None, None

In [7]:
# 7. Построение и сохранение индекса
print("Начинаем построение инвертированного индекса...")

# Проверяем структуру данных
print(f"Размер датасета: {len(df)} строк")
print(f"Колонки: {df.columns.tolist()}")
print("\nПервые 3 заголовка:")
for i in range(min(3, len(df))):
    print(f"{i}: {df.iloc[i]['title'][:100]}...")

Начинаем построение инвертированного индекса...
Размер датасета: 21673 строк
Колонки: ['N', 'source', 'rubric', 'title', 'text']

Первые 3 заголовка:
0: Синий богатырь...
1: Загитова согласилась вести «Ледниковый период»...
2: Объяснена опасность однообразного питания...


In [8]:
# Строим индекс
inverted_index, doc_lengths = build_inverted_index(df)

In [9]:
# Сохраняем индекс
save_index(inverted_index, doc_lengths)

Индекс сохранен в ../shared_data/search_index.pkl


In [11]:
# 8. Статистика индекса
print(f"\n=== СТАТИСТИКА ИНДЕКСА ===")
print(f"Количество документов: {len(df)}")
print(f"Количество уникальных терминов: {len(inverted_index)}")
print(f"Размер индекса в памяти: {sum(len(docs) for docs in inverted_index.values())} записей")



=== СТАТИСТИКА ИНДЕКСА ===
Количество документов: 21673
Количество уникальных терминов: 104125
Размер индекса в памяти: 2718491 записей


In [16]:
# Топ-10 самых частых терминов
term_doc_freq = {term: len(docs) for term, docs in inverted_index.items()}
top_terms = sorted(term_doc_freq.items(), key=lambda x: x[1], reverse=True)[:10]
print(f"\nТоп-10 самых частых терминов:")
for term, freq in top_terms:
    print(f"  {term}: встречается в {freq} документах")


Топ-10 самых частых терминов:
  новост: встречается в 16021 документах
  ри: встречается в 15198 документах
  котор: встречается в 13538 документах
  москв: встречается в 11071 документах
  росс: встречается в 9879 документах
  год: встречается в 9781 документах
  эт: встречается в 9757 документах
  такж: встречается в 8984 документах
  дан: встречается в 8472 документах
  ран: встречается в 7387 документах
